# ML experimentation Notebook

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

In [ ]:
pd.set_option("display.max_rows", None)

In [ ]:
file_path = "../data/ml_dataset.pkl"
df = pd.read_pickle(file_path)
print(f"DataFrame loaded with {len(df)} rows and {len(df.columns)} columns.")

DataFrame loaded with 98355 rows and 152 columns.


In [15]:
df.dtypes

tourney_id                                    object
tourney_date                          datetime64[ns]
match_num                                      int64
player_1                                       int64
player_2                                       int64
winner                                         int64
tourney_name                                  object
surface                                       object
draw_size                                    float64
tourney_level                                 object
tourney_year                                   int32
player_id                                      int64
player_seed                                  float64
player_entry                                  object
player_name                                   object
player_hand                                   object
player_ht                                    float64
player_ioc                                    object
player_age                                   f

In [ ]:
# Get all columns with dtype 'object'
cat_columns = df.select_dtypes(include="object").columns.tolist()
print(f"Categorical columns: {cat_columns}")

Categorical columns: ['tourney_id', 'tourney_name', 'surface', 'tourney_level', 'player_entry', 'player_name', 'player_hand', 'player_ioc', 'opponent_entry', 'opponent_name', 'opponent_hand', 'opponent_ioc', 'score', 'round', 'tourney_name_p2', 'surface_p2', 'tourney_level_p2', 'player_entry_p2', 'player_name_p2', 'player_hand_p2', 'player_ioc_p2', 'opponent_entry_p2', 'opponent_name_p2', 'opponent_hand_p2', 'opponent_ioc_p2', 'score_p2', 'round_p2', 'tourney_name_t', 'surface_t', 'tourney_level_t']


In [ ]:
# Get all columns with dtype 'datetime64[ns]'
date_columns = df.select_dtypes(include="datetime64[ns]").columns.tolist()
print(f"Datetime columns: {date_columns}")

Datetime columns: ['tourney_date', 'tourney_date_t']


In [ ]:
features = [
    col
    for col in df.columns
    if col not in cat_columns
    and col not in date_columns
    and "_id" not in col
    and "p_" not in col
    and "o_" not in col
    and col != "winner"
    and "results" not in col
]
len(features)

39

In [63]:
features

['match_num',
 'player_1',
 'player_2',
 'draw_size',
 'tourney_year',
 'player_seed',
 'player_ht',
 'player_age',
 'player_rank',
 'player_rank_points',
 'opponent_seed',
 'opponent_ht',
 'opponent_age',
 'opponent_rank',
 'opponent_rank_points',
 'best_of',
 'minutes',
 'opponent_rank_points_mean_last5',
 'minutes_mean_last5',
 'minutes_sum_last5',
 'draw_size_p2',
 'tourney_year_p2',
 'player_seed_p2',
 'player_ht_p2',
 'player_age_p2',
 'player_rank_p2',
 'player_rank_points_p2',
 'opponent_seed_p2',
 'opponent_ht_p2',
 'opponent_age_p2',
 'opponent_rank_p2',
 'opponent_rank_points_p2',
 'best_of_p2',
 'minutes_p2',
 'opponent_rank_points_mean_last5_p2',
 'minutes_mean_last5_p2',
 'minutes_sum_last5_p2',
 'draw_size_t',
 'tourney_year_t']

In [ ]:
target_column = "winner"
seed = 42

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df[features], df[target_column], test_size=0.2, random_state=seed
)

In [ ]:
print((y_train.value_counts() / len(y_train)) * 100)
print((y_test.value_counts() / len(y_test)) * 100)

winner
0    50.247827
1    49.752173
Name: count, dtype: float64
winner
0    50.170301
1    49.829699
Name: count, dtype: float64


In [ ]:
params = {
    "max_depth": 6,
    "learning_rate": 0.1,
    "n_estimators": 100,
    "colsample_bytree": 0.5,
}

In [ ]:
xgb_clf = XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    enable_categorical=True,
    seed=seed,
    **params,
)

In [69]:
xgb_clf_model = xgb_clf.fit(X_train, y_train)
y_pred_train = xgb_clf_model.predict(X_train)
y_pred = xgb_clf_model.predict(X_test)

print(f"Train Accuracy: {accuracy_score(y_train, y_pred_train)}")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred)}")


Train Accuracy: 0.6853108637079965
Test Accuracy: 0.6539067663057292
